In [49]:
import pandas as pd
import json
import pandas as pd
import requests
import re
import numpy as np
from time import sleep

def read_all_sheets(fname='input/locations.xlsx'):
    all_data = pd.read_excel(fname, sheet_name=None)
    for continent in all_data:
        all_data[continent]['continent'] = continent
    return pd.concat(all_data.values())


def parse_page(url: str):
    snowpack = {}
    measurement_date = {}

    page = requests.get(url)
    lines = (l.decode("utf-8") for l in page.iter_lines())
    snowfalls_pattern = re.compile('jssnowfalls([0-9]+)\s*=\s*(\[.*\])')
    dates_pattern = re.compile('jsdates([0-9]+)\s*=\s*(\[.*\])')
    
    for line in lines:
        snowfalls_match = snowfalls_pattern.findall(line)
        dates_match = dates_pattern.findall(line)
        if snowfalls_match:
            assert len(snowfalls_match)==1
            text_match = snowfalls_match[0]
            year = int(text_match[0])
            data = json.loads(text_match[1])
            snowpack[year] = data
        if dates_match:
            assert len(dates_match)==1
            text_match = dates_match[0]
            year = int(text_match[0])
            data = json.loads(text_match[1])
            measurement_date[year] = data
    return snowpack, measurement_date
    

def dicts_to_df(snow_data, dates, data_type):
    yearly_dfs = []
    for year in snow_data.keys():
        df = pd.DataFrame(zip(snow_data[year], dates[year]),
                          columns=[data_type, 'date'])
        df['season'] = year
        yearly_dfs.append(df)
    return pd.concat(yearly_dfs)

def get_one_resort(continent: str, region: str, resort: str):
    
    resort_base_url = "/".join(["https://www.onthesnow.com", region, resort, "historical-snowfall.html?y=0"]) # y=0 gets multiple years in one call
    snowpack_url = resort_base_url + "&q=top"
    snowfall_url = resort_base_url + "&q=snow"    
    snowpack, snowpack_dates = parse_page(snowpack_url)
    snowpack_df = dicts_to_df(snowpack, snowpack_dates, "snowpack")
    snowpack_df = snowpack_df[snowpack_df.snowpack>0]
    snowpack_df = snowpack_df[snowpack_df.snowpack<400]
    snowfall, snowfall_dates = parse_page(snowfall_url)
    snowfall_df = dicts_to_df(snowfall, snowfall_dates, "snowfall")
    for df in (snowpack_df, snowfall_df):
        df['continent'] = continent
        df['region'] = region
        df['resort'] = resort
        df['date'] = pd.to_datetime(df.date)
        df['day_of_year'] = df.date.dt.dayofyear
    return {'snowpack': snowpack_df, 'snowfall': snowfall_df}

def stochastic_sleep(min, max):
    sleep(np.random.uniform(min, max))

In [52]:
snow_data_accumulator = []

locations = read_all_sheets()

for index, row_data in locations.head(10).iterrows():
    snow_data_accumulator.append(get_one_resort(row_data.continent, row_data.Region, row_data.Resort))
    stochastic_sleep(1, 2)

snowpack = pd.concat(resort_data['snowpack'] for resort_data in snow_data_accumulator)
snow_fall = pd.concat(resort_data['snowfall'] for resort_data in snow_data_accumulator)

In [53]:
snowpack.to_csv('output/snowpack.csv', index=False)
snow_fall.to_csv('output/snow_fall.csv', index=False)

In [8]:
import altair as alt
alt.Chart(snowpack[snowpack.season==2015]).mark_line().encode(
    alt.X('date'),
    alt.Y('snowpack'),
    alt.Color('season:N')
)

alt.Chart(...)

In [14]:
breck_snowpack_chart = alt.Chart(breck_snowpack[breck_snowpack.day_of_year == 91]).mark_point().encode(
    alt.X('season:N'),
    alt.Y('snowpack', title='Snowpack on April 1')
)

breck_snowpack_chart + breck_snowpack_chart.transform_regression('season', 'snowpack').mark_line()

alt.LayerChart(...)